In [ ]:
import os
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [ ]:
csv_dir = '../../SEC Trackman Data/'
all_files = [os.path.join(csv_dir, f) for f in os.listdir(csv_dir) if f.endswith('.csv')]

df_list = []

for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)

all_pitches = pd.concat(df_list, ignore_index=True)

print(all_pitches.shape)
print(all_pitches.columns)

In [ ]:
ffs = all_pitches[all_pitches['TaggedPitchType'].isin(['FourSeamFastBall', 'Fastball'])]
print(ffs.shape)

In [ ]:
ffs['whiff'] = np.where(ffs['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

In [ ]:
ffs_X = ffs[preds]
ffs_y = ffs['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ffs_X, ffs_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_ffs = ffs[ffs['PitcherTeam']=='AUB_TIG']

In [ ]:
# Get unique values
unique_values = ffs['Pitcher'].unique()
aub_unique_values = aub_ffs['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: ffs[ffs['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: ffs[ffs['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_ffs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(ffs_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} Four Seam stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_ffs_stuff(pitcher)

In [ ]:
aub_subsets['Keplinger, Konner']

In [ ]:
ffs_rr = ffs[(ffs['PitcherThrows'] == 'Right') & (ffs['BatterSide'] == 'Right')]
ffs_rl = ffs[(ffs['PitcherThrows'] == 'Right') & (ffs['BatterSide'] == 'Left')]
ffs_lr = ffs[(ffs['PitcherThrows'] == 'Left') & (ffs['BatterSide'] == 'Right')]
ffs_ll = ffs[(ffs['PitcherThrows'] == 'Left') & (ffs['BatterSide'] == 'Left')]

print(ffs_rr.shape)
print(ffs_rl.shape)
print(ffs_lr.shape)
print(ffs_ll.shape)

In [ ]:
ffs_rr['whiff'] = np.where(ffs_rr['PitchCall']=='StrikeSwinging',1,0)
ffs_rl['whiff'] = np.where(ffs_rl['PitchCall']=='StrikeSwinging',1,0)
ffs_lr['whiff'] = np.where(ffs_lr['PitchCall']=='StrikeSwinging',1,0)
ffs_ll['whiff'] = np.where(ffs_ll['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

In [ ]:
ffs_rr_X = ffs_rr[preds]
ffs_rr_y = ffs_rr['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ffs_rr_X, ffs_rr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
y_pred_prob = model.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, predictions, multi_class='ovr')

print(f'ROC AUC: {roc_auc:.4f}')

# Plotting ROC Curve for one class (e.g., class 0)
fpr, tpr, _ = roc_curve(y_test == 1, y_pred_prob)
plt.plot(fpr, tpr, label='Class 0 ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='best')
plt.show()

In [ ]:
aub_ffs_rr = ffs_rr[ffs_rr['PitcherTeam']=='AUB_TIG']
print(aub_ffs_rr.shape)

In [ ]:
# Get unique values
unique_values = ffs_rr['Pitcher'].unique()
aub_unique_values = aub_ffs_rr['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: ffs_rr[ffs_rr['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: ffs_rr[ffs_rr['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rr_ffs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(ffs_rr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} RR Four Seam stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_rr_ffs_stuff(pitcher)

In [ ]:
ffs_rl_X = ffs_rl[preds]
ffs_rl_y = ffs_rl['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ffs_rl_X, ffs_rl_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_ffs_rl = ffs_rl[ffs_rl['PitcherTeam']=='AUB_TIG']
print(aub_ffs_rl.shape)

In [ ]:
# Get unique values
unique_values = ffs_rl['Pitcher'].unique()
aub_unique_values = aub_ffs_rl['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: ffs_rl[ffs_rl['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: ffs_rl[ffs_rl['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rl_ffs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(ffs_rl_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} Four Seam RL stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_rl_ffs_stuff(pitcher)

In [ ]:
ffs_lr_X = ffs_lr[preds]
ffs_lr_y = ffs_lr['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ffs_lr_X, ffs_lr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_ffs_lr = ffs_lr[ffs_lr['PitcherTeam']=='AUB_TIG']
print(aub_ffs_lr.shape)

In [ ]:
# Get unique values
unique_values = ffs_lr['Pitcher'].unique()
aub_unique_values = aub_ffs_lr['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: ffs_lr[ffs_lr['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: ffs_lr[ffs_lr['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_lr_ffs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(ffs_lr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} Four Seam LR stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_lr_ffs_stuff(pitcher)

In [ ]:
ffs_ll_X = ffs_ll[preds]
ffs_ll_y = ffs_ll['whiff']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ffs_ll_X, ffs_ll_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
aub_ffs_ll = ffs_ll[ffs_ll['PitcherTeam']=='AUB_TIG']
print(aub_ffs_ll.shape)

In [ ]:
# Get unique values
unique_values = ffs_ll['Pitcher'].unique()
aub_unique_values = aub_ffs_ll['Pitcher'].unique()

print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: ffs_ll[ffs_ll['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: ffs_ll[ffs_ll['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_ll_ffs_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(ffs_ll_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} Four Seam LL stuff+: {np.mean(stuff * 100)}")

In [ ]:
for pitcher in aub_unique_values:
    aub_ll_ffs_stuff(pitcher)

In [ ]:
aub_subsets['Vail, Tommy']